In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import re
import os
import pandas as pd
from urllib.parse import urlparse


# NOVO

In [ ]:
# OK
driver = webdriver.Chrome()
driver.implicitly_wait(15)

url = 'https://www.news.com.au/'
driver.get(url)

xpath_searchButton = '/html/body/nav/form/button'
xpath_searchBox = '/html/body/nav/form/input'

xpath_filter_tab = '//*[@id="refine"]'

xpath_type_all = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[1]/label'
xpath_type_audio = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[3]/label'
xpath_type_video = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[5]/label'
xpath_type_article = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[2]/label'
xpath_type_gallery = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[4]/label'

xpath_time_anytime = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[1]/label'
xpath_time_past_day = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[2]/label'
xpath_time_past_week = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[3]/label'
xpath_time_past_month = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[4]/label'
xpath_time_past_year = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[5]/label'

query = 'healthcare'

search_button = driver.find_element(By.XPATH, xpath_searchButton)
search_button.click()
print("Botão de busca clicado!")

search_box = driver.find_element(By.XPATH, xpath_searchBox)
search_box.send_keys(query)
search_box.send_keys(Keys.RETURN)
print("Pesquisa enviada!")
time.sleep(5)

if driver.find_element(By.XPATH, xpath_filter_tab):
    filter_tab = driver.find_element(By.XPATH, xpath_filter_tab)
    filter_tab.click()

filter_tab = driver.find_element(By.XPATH, xpath_type_article)
filter_tab.click()
time.sleep(3)
filter_tab = driver.find_element(By.XPATH, xpath_time_past_month)
filter_tab.click()
print('Começando as pesquisas........\n--------------')
time.sleep(3)

if driver.find_element(By.XPATH, xpath_filter_tab):
    filter_tab = driver.find_element(By.XPATH, xpath_filter_tab)
    filter_tab.click()

time.sleep(2)
# Localiza todos os artigos na página
articles = driver.find_elements(By.CLASS_NAME, 'storyblock')

# Define as frases de busca
search_phrases = [query]  # Exemplo de frases de busca

# Cria uma pasta para armazenar as imagens
os.makedirs('news_images', exist_ok=True)

# Lista para armazenar os dados de todos os artigos
articles_data = []

# Itera sobre cada artigo e extrai os dados
for article in articles:
    # Extrai o título
    title_element = article.find_element(By.CLASS_NAME, 'storyblock_title')
    title = title_element.text

    # Extrai a data
    date_element = article.find_element(By.CLASS_NAME, 'storyblock_datetime')
    date = date_element.get_attribute('datetime')

    # Extrai a descrição, se disponível
    description_element = article.find_elements(By.CLASS_NAME, 'storyblock_standfirst')
    description = description_element[0].text if description_element else None

    # Extrai o nome do arquivo da imagem
    image_element = article.find_element(By.CLASS_NAME, 'responsive-img_img')
    image_url = image_element.get_attribute('src')

    # Faz uma requisição para obter os dados da imagem
    response = requests.get(image_url)
    
    # Determina a extensão do arquivo
    content_type = response.headers.get('Content-Type')
    if 'image/jpeg' in content_type:
        extension = '.jpg'
    elif 'image/png' in content_type:
        extension = '.png'
    else:
        extension = ''  # Caso algum outro formato seja encontrado
    
    # Extrai o nome do arquivo da URL
    parsed_url = urlparse(image_url)
    image_filename = os.path.join('news_images', os.path.basename(parsed_url.path) + extension)

    # Salva a imagem
    with open(image_filename, 'wb') as handler:
        handler.write(response.content)

    # Conta as frases de busca no título e na descrição
    phrase_count = sum(title.lower().count(phrase.lower()) for phrase in search_phrases) + \
                   (sum(description.lower().count(phrase.lower()) for phrase in search_phrases) if description else 0)

    # Verifica se o título ou descrição contém algum valor monetário
    money_pattern = re.compile(r'(\$\d+(\.\d{1,2})?)|(\d+\s?(dollars|USD))')
    contains_money = bool(money_pattern.search(title) or (description and money_pattern.search(description)))

    # Armazena os dados do artigo
    articles_data.append({
        "Title": title,
        "Date": date,
        "Description": description,
        "Picture Filename": image_filename,
        "Count of Search Phrases": phrase_count,
        "Contains Money": contains_money
    })

    time.sleep(1)

# Exibe os resultados de todos os artigos
for article_data in articles_data:
    print(article_data)
print('Pesquisa finalizada')

# Cria um DataFrame com os dados coletados
df = pd.DataFrame(articles_data)

# Salva o DataFrame em um arquivo Excel
df.to_excel('news_data.xlsx', index=False)

# Fecha o navegador
driver.quit()

print("Processo concluído. Dados salvos em 'news_data.xlsx' e imagens baixadas em 'news_images/'.")


# *CLASSES*

In [1]:
import os
import re
import configparser
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from urllib.parse import urlparse
from time import sleep
import urllib.request
from PIL import Image
from io import BytesIO

# Configuração de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class NewsScraper:
    def __init__(self, config_file='config.ini'):
        self.config = configparser.ConfigParser()
        self.config.read(config_file)
        self.query = self.config.get('search', 'query')
        self.category = self.config.get('filters', 'category')
        self.time = int(self.config.get('filters', 'time'))
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(15)

    def search_news(self):
        url = 'https://www.news.com.au/'
        self.driver.get(url)

        try:
            search_button = self.driver.find_element(By.XPATH, '/html/body/nav/form/button')
            search_button.click()
            logging.info("Search button clicked.")

            search_box = self.driver.find_element(By.XPATH, '/html/body/nav/form/input')
            search_box.send_keys(self.query)
            search_box.send_keys(Keys.RETURN)
            logging.info("Search submitted.")
            sleep(5)

            filter_tab = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="refine"]'))
            )
            filter_tab.click()

            if self.category == 'article':
                category_xpath = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[2]/label'
            elif self.category == 'audio':
                category_xpath = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[3]/label'
            elif self.category == 'gallery':
                category_xpath = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[4]/label'
            elif self.category == 'video':
                category_xpath = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[5]/label'
            else:
                category_xpath = '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[1]/label'  # Default to 'all'

            category_filter = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, category_xpath))
            )
            category_filter.click()
            logging.info(f"Category filter '{self.category}' applied.")

            if self.time == 'day':
                time_xpath = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[2]/label'

            elif self.time == 'week':
                time_xpath = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[3]/label'

            elif self.time == 'month':
                time_xpath = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[4]/label'

            elif self.time == 'year':
                time_xpath = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[5]/label'

            else:
                time_xpath = '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[1]/label'

            time_filter = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, time_xpath))
            )
            time_filter.click()
            logging.info(f"Time filter applied for the last {self.time}.")
            sleep(3)

            filter_tab.click()
            sleep(2)

        except Exception as e:
            logging.error(f"Error applying the filters: {e}")
            self.driver.quit()
            raise

    def download_and_save_image(image_url, filename, format='JPEG'):
        try:
            # Baixa a imagem da URL
            with urllib.request.urlopen(image_url) as response:
                image_data = response.read()
            
            # Abre a imagem em memória
            image = Image.open(BytesIO(image_data))
            
            # Salva a imagem com o nome e formato fornecidos
            os.makedirs(os.path.dirname(filename), exist_ok=True)  # Certifica-se de que o diretório existe
            image.save(filename, format=format)  # Salva a imagem com o formato especificado
            
            print(f"Image save as '{filename}'")
            
        except Exception as e:
            print(f"Erro ao baixar ou salvar a imagem: {e}")

    def process_articles(self):
        articles = self.driver.find_elements(By.CLASS_NAME, 'storyblock')
        os.makedirs('news_images', exist_ok=True)
        articles_data = []

        for article in articles:
            try:
                title_element = article.find_element(By.CLASS_NAME, 'storyblock_title')
                title = title_element.text

                date_element = article.find_element(By.CLASS_NAME, 'storyblock_datetime')
                date = date_element.get_attribute('datetime')

                description_element = article.find_elements(By.CLASS_NAME, 'storyblock_standfirst')
                description = description_element[0].text if description_element else None

                image_element = article.find_element(By.CLASS_NAME, 'responsive-img_img')
                image_url = image_element.get_attribute('src')

                # response = requests.get(image_url)
                # content_type = response.headers.get('Content-Type')

                # extension = '.jpg' if 'jpeg' in content_type else '.png' if 'png' in content_type else ''
                parsed_url = urlparse(image_url)
                image_filename = os.path.join('news_images', os.path.basename(parsed_url.path) + ".jpeg")

                # with open(image_filename, 'wb') as handler:
                #     handler.write(response.content)

                download_and_save_image(image_url, image_filename)

                search_phrases = [self.query]
                phrase_count = sum(title.lower().count(phrase.lower()) for phrase in search_phrases) + \
                               (sum(description.lower().count(phrase.lower()) for phrase in search_phrases) if description else 0)

                money_pattern = re.compile(r'(\$\d+(\.\d{1,2})?)|(\d+\s?(dollars|USD))')
                contains_money = bool(money_pattern.search(title) or (description and money_pattern.search(description)))

                articles_data.append({
                    "Title": title,
                    "Date": date,
                    "Description": description,
                    "Picture Filename": image_filename,
                    "Count of Search Phrases": phrase_count,
                    "Contains Money": contains_money
                })

                sleep(1)

            except Exception as e:
                logging.warning(f"Error processing the article: {e}")

        return articles_data

    def save_to_excel(self, articles_data):
        df = pd.DataFrame(articles_data)
        df.to_excel('news_data.xlsx', index=False)
        logging.info("Data saved in 'news_data.xlsx'.")

    def run(self):
        try:
            self.search_news()
            articles_data = self.process_articles()
            self.save_to_excel(articles_data)
        except Exception as e:
            logging.error(f"Error in the scraping process: {e}")
        finally:
            self.driver.quit()

if __name__ == "__main__":
    scraper = NewsScraper()
    scraper.run()
    logging.info("Process completed.")


2024-08-14 17:44:48,122 - INFO - Botão de busca clicado!
2024-08-14 17:44:58,451 - INFO - Pesquisa enviada!
2024-08-14 17:45:03,680 - INFO - Filtro de categoria 'article' aplicado!
2024-08-14 17:45:03,788 - INFO - Filtro de tempo aplicado para os últimos 3 meses!
2024-08-14 17:45:36,189 - INFO - Dados salvos em 'news_data.xlsx'.
2024-08-14 17:45:38,621 - INFO - Processo concluído.


____________________

In [2]:
import os
import re
import configparser
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from urllib.parse import urlparse
from time import sleep
import urllib.request
from PIL import Image
from io import BytesIO

# Configuração de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class NewsScraper:
    def __init__(self, config_file='config.ini'):
        self.config = configparser.ConfigParser()
        self.config.read(config_file)
        self.query = self.config.get('search', 'query')
        self.category = self.config.get('filters', 'category')
        self.time = self.config.get('filters', 'time')  # Mantenha como string se as comparações forem com strings
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(15)

    def search_news(self):
        url = 'https://www.news.com.au/'
        self.driver.get(url)

        try:
            search_button = self.driver.find_element(By.XPATH, '/html/body/nav/form/button')
            search_button.click()
            logging.info("Search button clicked.")

            search_box = self.driver.find_element(By.XPATH, '/html/body/nav/form/input')
            search_box.send_keys(self.query)
            search_box.send_keys(Keys.RETURN)
            logging.info("Search submitted.")
            sleep(5)

            filter_tab = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="refine"]'))
            )
            filter_tab.click()

            category_xpath_map = {
                'article': '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[2]/label',
                'audio': '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[3]/label',
                'gallery': '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[4]/label',
                'video': '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[5]/label',
            }
            category_xpath = category_xpath_map.get(self.category, '//*[@id="custom-facet-0"]/div/div[2]/div/ul/li[1]/label')

            category_filter = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, category_xpath))
            )
            category_filter.click()
            logging.info(f"Category filter '{self.category}' applied.")

            time_xpath_map = {
                'day': '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[2]/label',
                'week': '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[3]/label',
                'month': '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[4]/label',
                'year': '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[5]/label',
            }
            time_xpath = time_xpath_map.get(self.time, '//*[@id="custom-facet-1"]/div/div[2]/div/ul/li[1]/label')

            time_filter = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, time_xpath))
            )
            time_filter.click()
            logging.info(f"Time filter applied for the last {self.time}.")
            sleep(3)

            filter_tab.click()
            sleep(2)

        except Exception as e:
            logging.error(f"Error applying the filters: {e}")
            self.driver.quit()
            raise

    def download_and_save_image(self, image_url, filename, format='JPEG'):
        try:
            # Baixa a imagem da URL
            with urllib.request.urlopen(image_url) as response:
                image_data = response.read()
            
            # Abre a imagem em memória
            image = Image.open(BytesIO(image_data))
            
            # Salva a imagem com o nome e formato fornecidos
            os.makedirs(os.path.dirname(filename), exist_ok=True)  # Certifica-se de que o diretório existe
            image.save(filename, format=format)  # Salva a imagem com o formato especificado
            
            logging.info(f"Image saved as '{filename}'.")
            
        except Exception as e:
            logging.error(f"Error downloading or saving the image: {e}")

    def process_articles(self):
        articles = self.driver.find_elements(By.CLASS_NAME, 'storyblock')
        os.makedirs('news_images', exist_ok=True)
        articles_data = []

        for article in articles:
            try:
                title_element = article.find_element(By.CLASS_NAME, 'storyblock_title')
                title = title_element.text

                date_element = article.find_element(By.CLASS_NAME, 'storyblock_datetime')
                date = date_element.get_attribute('datetime')

                description_element = article.find_elements(By.CLASS_NAME, 'storyblock_standfirst')
                description = description_element[0].text if description_element else None

                image_element = article.find_element(By.CLASS_NAME, 'responsive-img_img')
                image_url = image_element.get_attribute('src')

                parsed_url = urlparse(image_url)
                image_filename = os.path.join('news_images', os.path.basename(parsed_url.path) + ".jpeg")

                self.download_and_save_image(image_url, image_filename)

                search_phrases = [self.query]
                phrase_count = sum(title.lower().count(phrase.lower()) for phrase in search_phrases) + \
                               (sum(description.lower().count(phrase.lower()) for phrase in search_phrases) if description else 0)

                money_pattern = re.compile(r'(\$\d+(\.\d{1,2})?)|(\d+\s?(dollars|USD))')
                contains_money = bool(money_pattern.search(title) or (description and money_pattern.search(description)))

                articles_data.append({
                    "Title": title,
                    "Date": date,
                    "Description": description,
                    "Picture Filename": image_filename,
                    "Count of Search Phrases": phrase_count,
                    "Contains Money": contains_money
                })

                sleep(1)

            except Exception as e:
                logging.warning(f"Error processing the article: {e}")

        return articles_data

    def save_to_excel(self, articles_data):
        df = pd.DataFrame(articles_data)
        df.to_excel('news_data.xlsx', index=False)
        logging.info("Data saved in 'news_data.xlsx'.")

    def run(self):
        try:
            self.search_news()
            articles_data = self.process_articles()
            self.save_to_excel(articles_data)
        except Exception as e:
            logging.error(f"Error in the scraping process: {e}")
        finally:
            self.driver.quit()

if __name__ == "__main__":
    scraper = NewsScraper()
    scraper.run()
    logging.info("Process completed.")


2024-08-14 20:17:44,715 - INFO - Search button clicked.
2024-08-14 20:17:50,645 - INFO - Search submitted.
2024-08-14 20:17:55,879 - INFO - Category filter 'article' applied.
2024-08-14 20:17:55,981 - INFO - Time filter applied for the last month.
2024-08-14 20:18:01,583 - INFO - Image saved as 'news_images\2584f646b4d713d5bf42b404d8ccc6a1.jpeg'.
2024-08-14 20:18:02,817 - INFO - Image saved as 'news_images\09dd5bc00ca4d679b00524a7dafe6157.jpeg'.
2024-08-14 20:18:04,101 - INFO - Image saved as 'news_images\fe8a23f9e06e4b532946f00f167dbb95.jpeg'.
2024-08-14 20:18:05,531 - INFO - Image saved as 'news_images\3edeae1dbc8c18cf4dfb6343049dea1f.jpeg'.
2024-08-14 20:18:06,907 - INFO - Image saved as 'news_images\26e3c79087156e34b58da64fa911883e.jpeg'.
2024-08-14 20:18:08,242 - INFO - Image saved as 'news_images\1f2eb21b595010b5daab36579b483d6a.jpeg'.
2024-08-14 20:18:09,616 - INFO - Image saved as 'news_images\c9ae12c82a6aaafe6b1ada577bd3d080.jpeg'.
2024-08-14 20:18:10,984 - INFO - Image saved 